### 1 - Import libs

In [1]:
import time 
from pathlib import Path
from datetime import datetime as dt

import ee
import geemap
# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import helpers as h
import geopandas as gpd

/usr/local/lib/python3.8/dist-packages/bfast/__init__.py:10: UserWarning: cupy is not available in this environment, GPU fonctionnalities won't be available
  warn("cupy is not available in this environment, GPU fonctionnalities won't be available")


### 2 - Basic Input Variables

In [2]:
# Area of Interest
country = 'Kenya'   # country case
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', country)) # here any feature collection can be selected

# Grid Size (in degrees)
grid_size = 0.5

### 3 - Generate GRID and POINTS

In [3]:
# Generate the grid
grid = h.generate_grid(aoi, grid_size, grid_size, 0, 0).filterBounds(aoi)

# get a random point over each grid cell
randomInCell = grid.map(h.getRandomPoint)

# get the centroid of each gridcell
centerInCell = grid.map(h.getCenterPoint)

# stratified sampling based on ESA LC
imageCollection = ee.ImageCollection("ESA/WorldCover/v100")
esaLc = imageCollection.filterBounds(aoi).first().clip(aoi)

stratifiedSamples = esaLc.stratifiedSample(**{
    "numPoints": 5,
    "region": aoi,
    "scale": 100, 
    "seed": 42, 
    "tileScale": 4, 
    "geometries": True
}).map(h.setId)

### 4 - Display in map

In [ ]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.centerObject(aoi,6)

Map.addLayer(aoi)
Map.addLayer(grid)
Map.addLayer(centerInCell, {}, '1 Center Sample per gridcell (Systematic)')

#Map.addLayer(randomInCell, {}, '1 Random Sample per gridcell (Systematic random)')
#Map.addLayer(esaLc, null, 'ESA Worldcover')
#Map.addLayer(stratifiedSamples, {}, 'Stratified Samples per class from ESA LC')

Map

### 5 - Add auxillary data from global datasets

In [6]:
## Global Forest Change (Hansen et al., 2013)
gfc_col = ee.Image('UMD/hansen/global_forest_change_2020_v1_8').select(['treecover2000','loss','lossyear','gain'],['gfc_tc00','gfc_loss','gfc_year','gfc_gain'])

## ESA WorldCover 2020
esa_20  = ee.Image('ESA/WorldCover/v100/2020').rename('esa_lc20')

## Tropical Moist Forest - JRC 2021
tmf_annual= ee.ImageCollection('projects/JRC/TMF/v1_2020/AnnualChanges').mosaic()
tmf_ann_n = tmf_annual.rename(tmf_annual.bandNames().map(h.rename_TMF))

tmf_subtp = ee.ImageCollection('projects/JRC/TMF/v1_2020/TransitionMap_Subtypes').mosaic().rename('tmf_subtypes')
tmf_main  = ee.ImageCollection('projects/JRC/TMF/v1_2020/TransitionMap_MainClasses').mosaic().rename('tmf_main_cl')
tmf_deg   = ee.ImageCollection('projects/JRC/TMF/v1_2020/DegradationYear').mosaic().rename('tmf_deg_yr')
tmf_def   = ee.ImageCollection('projects/JRC/TMF/v1_2020/DeforestationYear').mosaic().rename('tmf_def_yr')

##  COMBINE COLLECTIONS
glo_ds = esa_20.addBands(gfc_col).addBands(tmf_subtp).addBands(tmf_main).addBands(tmf_deg).addBands(tmf_def).addBands(tmf_annual_n)

## EXTRACT THE DATA TO THE POINTS
centerPtsAux = glo_ds.reduceRegions(**{
  "reducer": ee.Reducer.first(),
  "collection": centerInCell
}).select(['point_id','esa_lc20','gfc_tc00','gfc_loss','gfc_year','gfc_gain',
           'tmf_main_cl','tmf_subtypes','tmf_1990','tmf_1995','tmf_2000','tmf_2005','tmf_2010','tmf_2015','tmf_2020','tmf_def_yr','tmf_deg_yr',
           '.geo']);

### 6 EXPORT AS TABLE

In [5]:
## MAKE AS A DATAFRAME AND EXPORT
json = centerPtsAux.getInfo()
df   = gpd.GeoDataFrame.from_features(json)
df

,geometry,esa_lc20,gfc_gain,gfc_loss,gfc_tc00,gfc_year,point_id,tmf_1990,tmf_1995,tmf_2000,tmf_2005,tmf_2010,tmf_2015,tmf_2020,tmf_def_yr,tmf_deg_yr,tmf_main_cl,tmf_subtypes
0,POINT (33.66018 0.02589),80,0,0,0,NaN,33,5,5,5,5,5,5,5,0,0,60,71
1,POINT (34.16018 -0.97410),30,0,0,1,NaN,53,6,6,6,6,6,6,6,0,0,70,91
2,POINT (34.16018 -0.47410),80,0,0,0,NaN,54,5,5,5,5,5,5,5,0,0,60,71
3,POINT (34.16018 0.02589),30,0,0,12,NaN,55,6,6,6,6,6,6,6,0,0,70,91
4,POINT (34.16018 0.52589),30,0,0,16,NaN,56,6,6,6,6,6,6,6,0,0,70,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,POINT (41.66018 -1.97410),80,0,0,0,NaN,381,5,5,5,5,5,5,5,0,0,60,72
227,POINT (41.66018 -1.47410),20,0,0,12,NaN,382,6,6,6,6,6,6,6,0,0,70,91
228,POINT (41.66018 3.02589),20,0,0,0,NaN,391,6,6,6,6,6,6,6,0,0,70,91
229,POINT (41.66018 3.52588),20,0,0,0,NaN,392,6,6,6,6,6,6,6,0,0,70,91


In [ ]:
################# TO DO
## SPLIT geometry into LAT and LON
## REORDER with LON LAT PLOTID as first 3 COLUMNS

df.to_csv("/home/dannunzio/kenya/data/points.csv")
